In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from tqdm.notebook import tqdm
import pandas as pd
import requests

In [75]:
url = 'https://nyheder.tv2.dk/seneste'
driver = webdriver.Chrome()
driver.get(url)

headers = {
    'Name': 'Mathias Udsen', 
    'E-mail':'lcv842@alumni.ku.dk',
    'Message': 'Scraping as a part of our exam project at UCPH, Intoduction to social data science' 
}
requests.get(url, headers)
time.sleep(5)

<Response [200]>

In [76]:
cookie = driver.find_element(By.CSS_SELECTOR, '#onetrust-reject-all-handler')
cookie.click()
time.sleep(5)

In [33]:
Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
Vis_flere.click()
time.sleep(5)

In [5]:
"""
i = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')
        Vis_flere.click()
        time.sleep(1)
        i = i + 1
    except:
        time.sleep(1)
"""

"\ni = 0\nwhile i <= 10:\n    try:\n        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__bound')\n        Vis_flere.click()\n        time.sleep(1)\n        i = i + 1\n    except:\n        time.sleep(1)\n"

In [79]:
"""
i = 0
count_of_one_column_lists = 0
while i <= 10:
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
        i += 1
    except:
        time.sleep(1)
"""

"\ni = 0\ncount_of_one_column_lists = 0\nwhile i <= 10:\n    try:\n        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')\n        Vis_flere.click()\n\n        new_count_of_one_column_lists = 0\n        while new_count_of_one_column_lists <= count_of_one_column_lists:\n            soup = BeautifulSoup(driver.page_source, 'html.parser')\n            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))\n            time.sleep(0.25)\n\n        count_of_one_column_lists = new_count_of_one_column_lists\n        i += 1\n    except:\n        time.sleep(1)\n"

In [77]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

Headline = []
Category = []
Authors = []
Time = []
Live = []
All_text = []
nummer=0

Live_find1 = False
Live_find2 = False

max_retries = 10
retries = 0

Mathias = 'Ja'

max_iter = 500
i = 500
count_of_one_column_lists = 0
if Mathias == 'Ja':
    
    pbar1 = tqdm(total=max_iter, desc="Loading content")

    while i < 1001:
        try:
            Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
            Vis_flere.click()

            new_count_of_one_column_lists = 0
            while new_count_of_one_column_lists <= count_of_one_column_lists:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
                time.sleep(0.25)

            count_of_one_column_lists = new_count_of_one_column_lists
            i += 1
            pbar1.update(1)
        except:
            time.sleep(1)
    pbar1.close()

pbar2 = tqdm(total=max_iter, desc="Scraping")
i = 0
for i in range(500, 1001, 1):
    try:
        Vis_flere = driver.find_element(By.CSS_SELECTOR, '.js-tc_load-more__trigger.tc_button--secondary.tc_button--default.js-tc_load-more__bound')
        Vis_flere.click()

        new_count_of_one_column_lists = 0
        while new_count_of_one_column_lists <= count_of_one_column_lists:
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            new_count_of_one_column_lists = len(soup.find_all('ul', {'data-columns': '1'}))
            time.sleep(0.25)

        count_of_one_column_lists = new_count_of_one_column_lists
    except:
        time.sleep(1)
        
    retries = 0
    while retries < max_retries:
        try:
            one_column_lists = soup.find_all('ul', {'data-columns': '1'})[i]
            break
        except:
            #print(i)
            retries = retries + 1
            time.sleep(1)
    
    if retries == max_retries:
        print(f"Max tries at: Number {nummer} and i: {i}")
        continue
        
    pbar3 = tqdm(total=len(soup.find_all('ul', {'data-columns': '1'})[i].find_all(class_='tc_teaser')), desc="Articles", leave=False)

    for one_column_list in one_column_lists:
        artikler = one_column_list.find_all(class_='tc_teaser')
            
        for artikel in artikler:
            nummer += 1

            # Initializations
            current_headline = None
            current_author = None
            current_time = None
            current_category = None
            current_text = []
            current_live = 0

            # Check for Live label
            if artikel.find('span', class_='tc_label tc_label--color-live'):
                Live_find1 = True
            else:
                Live_find1 = False

            # Extract link
            href = artikel.find('a', class_='tc_teaser__link').get('href')

            if href:
                driver.execute_script(f'window.open("{href}", "_blank");')
                driver.switch_to.window(driver.window_handles[-1])

                new_html = driver.page_source
                new_soup = BeautifulSoup(new_html, 'lxml')

                # Live blog check
                liveblog_body = new_soup.find('div', class_="tc_liveblog__body", attrs={'data-adobe-context': "article-body"})
                Live_find2 = bool(liveblog_body and liveblog_body.find('em') and liveblog_body.find('em').text == 'Denne liveblog opdateres ikke længere.')

                # Extract details
                if Live_find1 or Live_find2:
                    current_headline = new_soup.find('header', class_="tc_page__header").find('h1').text

                byline = new_soup.find('div', class_="tc_byline")
                if byline:
                    current_author = byline.find('strong').text
                timestamp = new_soup.find('time', class_='tc_timestamp')
                if timestamp:
                    current_time = timestamp.get('datetime')

                if liveblog_body:
                    article_body_div = liveblog_body
                else:
                    article_body_div = new_soup.find('div', class_='tc_page__body', attrs={'data-adobe-context': 'article-body'})

                if article_body_div:
                    rich_content_paragraphs = article_body_div.find_all('p')
                    for paragraph in rich_content_paragraphs:
                        current_text.append(paragraph.text)

                driver.close()
                driver.switch_to.window(driver.window_handles[0])

            if Live_find1 or Live_find2:
                current_live = 1
            else:
                current_headline = artikel.find('a', class_='tc_teaser__link').get('aria-label')

            label = artikel.find('span', class_='tc_label tc_label--color-nyheder')
            if label:
                current_category = label.text

            if all([current_headline, current_author, current_time, current_category, current_text]):
                Headline.append(current_headline)
                Authors.append(current_author)
                Time.append(current_time)
                Category.append(current_category)
                All_text.append(current_text)
                Live.append(current_live)
            else:
                print(f"Skipped article at nummer: {nummer} due to missing details: {href}")

            pbar3.update(1)
    pbar3.close()
    pbar2.update(1)
pbar2.close()
time.sleep(5)

Scraping:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/5 [00:00<?, ?it/s]

Articles:   0%|          | 0/19 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

...

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

Articles:   0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
print(len(Category), Category[0])
print(len(Authors), Authors[0])
print(len(Headline), Headline[0])
print(len(Time), Time[0])
print(len(Live), Live[0])
print(len(All_text), All_text[0])
time.sleep(5)

129 Lokalt
129  Pernille Leftes
129 Jysk skole opretter 0. klasse med bare tre elever
129 2023-08-14T11:04:48.740Z
129 0
129 ['Mange spændte børn har mandag første skoledag i 0. klasse. Også i Ørslevkloster, hvor 0. klasse i år består af kun tre nye elever.', 'Petrina, Anna og Olivia. ', 'Navneopråbet på de nye elever i 0. klasse på skolen i Ørslevkloster er hurtig overstået, for årgangen består af kun tre piger. Det er dog ikke noget, der bekymrer Annas forældre.', '- Vi er slet ikke utrygge, fordi det personale, der er her, er virkelig gode til børnene. Og fagligheden fejler ikke noget, og de er gode til at se hvert enkelt barn, siger Mette Vestergaard Kristensen. ', 'De tre nye skoleelever skal dog ikke nøjes med hinandens selskab. Deres årgang bliver fra start slået sammen med både 1. og 2. klasse, så de bliver 23 elever på holdet:', '- Så lærer de jo af de ældre børn. Jeg tror faktisk, at det er en god idé i forhold til skolestart, at de er sammen med nogle, der har gået lidt i sk

In [81]:
data = {
    'Category': Category,
    'Authors': Authors,
    'Headline': Headline,
    'Time': Time,
    'Live': Live,
    'All_text': All_text
}
df = pd.DataFrame(data)
time.sleep(5)
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
df
#sorted_df = df.sort_values(by='Time')
#sorted_df

Category  \
0                 Krigen i Ukraine   
1                           Lokalt   
2                         Business   
3                           Lokalt   
4                          Samfund   
5                          Politik   
6                          Nyheder   
7                           Lokalt   
8                           Lokalt   
9                           Udland   
10                         Samfund   
11                          Udland   
12                         Nyheder   
13                         Politik   
14                          Lokalt   
15                           Krimi   
16                           Krimi   
17                          Udland   
18                          Udland   
19                Krigen i Ukraine   
20                          Lokalt   
21                         Samfund   
22                          Udland   
23                          Lokalt   
24                Krigen i Ukraine   
25                          Udland   
26                         Politik   
27                          Lokalt   
28                          Lokalt   
29                         Politik   
30                          Lokalt   
31                         Samfund   
32                         Politik   
33                          Lokalt   
34                Krigen i Ukraine   
35                         Politik   
36                          Udland   
37                           Krimi   
38                          Udland   
39                         Samfund   
40                         Nyheder   
41                         Samfund   
42                         Politik   
43                Krigen i Ukraine   
44                         Samfund   
45                Krigen i Ukraine   
46                          Lokalt   
47                          Udland   
48                         Politik   
49                         Samfund   
50                         Politik   
51                         Samfund   
52                          Udland   
53                          Udland   
54                           Klima   
55                           Krimi   
56                          Udland   
57                          Udland   
58                         Politik   
59                          Lokalt   
60                          Lokalt   
61                           Krimi   
62                         Samfund   
63                          Udland   
64                         Samfund   
65                             GO'   
66                          Udland   
67                         Politik   
68                          Udland   
69                Krigen i Ukraine   
70                         Nyheder   
71                         Samfund   
72                          Lokalt   
73                          Udland   
74                          Udland   
75                         Politik   
76                Krigen i Ukraine   
77                          Lokalt   
78                          Lokalt   
79                          Lokalt   
80                Krigen i Ukraine   
81                         Nyheder   
82                          Lokalt   
83                         Nyheder   
84                          Lokalt   
85                          Lokalt   
86                         Nyheder   
87                          Udland   
88                         Politik   
89                          Udland   
90                Krigen i Ukraine   
91                         Nyheder   
92                          Udland   
93                           Krimi   
94                         Samfund   
95                          Udland   
96                        Business   
97                Krigen i Ukraine   
98                 Dagens overblik   
99                          Udland   
100               Krigen i Ukraine   
101               Krigen i Ukraine   
102                         Udland   
103                            GO'   
104                          Krimi  

In [10]:
df.to_csv('Data.csv', index=False)
time.sleep(5)

In [84]:
df.to_csv(r'C:\Users\Mathias\Desktop\Privat\Uni\~Kandidat\ISDS\Data.csv', index=False)
time.sleep(5)

In [17]:
import os
os.system("shutdown /s /t 10")

0

In [18]:
#os.system("shutdown /a")

0